In [1]:
import pandas as pd
import json

url_part1=('https://raw.githubusercontent.com/civicliteracies/EITI_SDT_data_verification_and_validation/sqlite/4_clean/2_data_editing/output/eiti-data_part1_1.3.csv')
url_part5=('https://raw.githubusercontent.com/civicliteracies/EITI_SDT_data_verification_and_validation/sqlite/4_clean/2_data_editing/output/eiti-data_part5-0.11.8.csv')

df_part1 = pd.read_csv(url_part1)
df_part5 = pd.read_csv(url_part5)

/var/folders/z_/k9wh63yn4bzb0fzdgjkcx6tw0000gn/T/ipykernel_96606/1677188447.py:8: DtypeWarning: Columns (5,8,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part5 = pd.read_csv(url_part5)


# Part 1 - Generating statements

In [2]:
mapping_statement = {
    'declaration_reference': df_part1['iso_alpha2_code'] + '-' + df_part1['start_date'].str.replace('-','') + '-' + df_part1['end_date'].str.replace('-',''),
    'declaration_subject': df_part1['country_or_area_name'],
    'bods_version': '0.4',
    'license_url': 'http://opendatacommons.org/licenses/pddl/1.0/',
    'publication_date': df_part1['end_date'],
    'publisher_name': 'Extractive Industries Transparency initiative',
    'publisher_url': 'https://eiti.org/open-data',
    'record_identifier': df_part1['eiti_id_declaration'],
    'record_type': '',
    'asserted_by': df_part1['submitter_organisation'],
    'submitter_name': df_part1['submitter_name'],
    'submitter_uri': df_part1['submitter_email'],
    'retrieved_at': pd.Timestamp('today').strftime('%Y-%m-%d'),
    'source_type': 'officialRegister, verified',
    'source_url': 'https://eiti.portaljs.com',
    'statement_date': df_part1['eiti_data_publication_date'],
    'statement_identifier': df_part1['eiti_id_declaration']
}

df_statement = pd.DataFrame(mapping_statement)


In [3]:
print(f"Number of rows in df_statement: {df_statement.shape[0]}")
display(df_statement.head())

Number of rows in df_statement: 73


,declaration_reference,declaration_subject,bods_version,license_url,publication_date,publisher_name,publisher_url,record_identifier,record_type,asserted_by,submitter_name,submitter_uri,retrieved_at,source_type,source_url,statement_date,statement_identifier
0,AF-20171221-20181220,Afghanistan,0.4,http://opendatacommons.org/licenses/pddl/1.0/,2018-12-20,Extractive Industries Transparency initiative,https://eiti.org/open-data,1f61bd83-c1cd-3658-8fab-29ba86d584a7,entity,NaN,NaN,NaN,2024-05-18,verified,https://eiti.portaljs.com,NaN,1f61bd83-c1cd-3658-8fab-29ba86d584a7
1,AF-20181221-20191220,Afghanistan,0.4,http://opendatacommons.org/licenses/pddl/1.0/,2019-12-20,Extractive Industries Transparency initiative,https://eiti.org/open-data,c40776d5-a273-3f9d-b805-075e804b9f3e,entity,NaN,NaN,NaN,2024-05-18,verified,https://eiti.portaljs.com,NaN,c40776d5-a273-3f9d-b805-075e804b9f3e
2,AL-20170101-20171231,Albania,0.4,http://opendatacommons.org/licenses/pddl/1.0/,2017-12-31,Extractive Industries Transparency initiative,https://eiti.org/open-data,fb121867-d9c5-3112-90f5-f798ed67c49d,entity,EITI Albania Secretariat,EITI Albania Secretariat,sekretariati@albeiti.gov.al,2024-05-18,verified,https://eiti.portaljs.com,NaN,fb121867-d9c5-3112-90f5-f798ed67c49d
3,AL-20180101-20181231,Albania,0.4,http://opendatacommons.org/licenses/pddl/1.0/,2018-12-31,Extractive Industries Transparency initiative,https://eiti.org/open-data,e821dd0c-7660-3334-a55a-732ab12351d7,entity,EITI Albania Secretariat,EITI Albania Secretariat,sekretariati@albeiti.gov.al,2024-05-18,verified,https://eiti.portaljs.com,NaN,e821dd0c-7660-3334-a55a-732ab12351d7
4,AR-20180101-20181231,Argentina,0.4,http://opendatacommons.org/licenses/pddl/1.0/,2018-12-31,Extractive Industries Transparency initiative,https://eiti.org/open-data,2df5c073-4216-3f74-a2cd-23aa44dd3c9c,entity,Consultor Intependiente,Santiago J. Dondo,sjdondo@gmail.com,2024-05-18,verified,https://eiti.portaljs.com,NaN,2df5c073-4216-3f74-a2cd-23aa44dd3c9c


In [4]:
bods_statement_structure = {
    "statementId": "",
    "statementDate": "",
    "annotations": [],
    "publicationDetails": {
        "publicationDate": "",
        "bodsVersion": "",
        "license": "",
        "publisher": {
            "name": "",
            "url": ""
        }
    },
    "source": {
        "type": [],
        "description": "",
        "url": "",
        "retrievedAt": "",
        "assertedBy": [
            {
                "name": "",
                "uri": ""
            }
        ]
    },
    "declaration": "",
    "declarationSubject": "",
    "recordId": "",
    "recordType": "",
    "recordDetails": {}
}

In [47]:
# Dictionary to hold the JSON strings
json_variables = {}

# Iterate over each row in df_statement
for index, row in df_statement.iterrows():
    bods_statement = bods_statement_structure.copy()
    

    bods_statement["statementId"] = row['statement_identifier']
    bods_statement["statementDate"] = row['statement_date']
    bods_statement["publicationDetails"]["publicationDate"] = row['publication_date']
    bods_statement["publicationDetails"]["bodsVersion"] = row['bods_version']
    bods_statement["publicationDetails"]["license"] = row['license_url']
    bods_statement["publicationDetails"]["publisher"]["name"] = row['publisher_name']
    bods_statement["publicationDetails"]["publisher"]["url"] = row['publisher_url']
    bods_statement["source"]["type"] = row['source_type'].split(', ')
    bods_statement["source"]["url"] = row['source_url']
    bods_statement["source"]["retrievedAt"] = row['retrieved_at']
    bods_statement["source"]["assertedBy"][0]["name"] = row['submitter_name']
    bods_statement["source"]["assertedBy"][0]["uri"] = row['submitter_uri']
    bods_statement["declaration"] = row['declaration_reference']
    bods_statement["declarationSubject"] = row['declaration_subject']
    bods_statement["recordId"] = row['record_identifier']
    bods_statement["recordType"] = row['record_type']

    # Convert to JSON
    bods_json = json.dumps(bods_statement, indent=2)
    
    # Create a variable name based on the statement identifier
    variable_name = row['statement_identifier']
    
    # Save the JSON string in the dictionary
    json_variables[variable_name] = bods_json


In [50]:
for var_name, json_str in json_variables.items():
    print(f"KEY: {var_name}")
    pretty_json = json.dumps(json.loads(json_str), indent=2)
    print(f"VALUE: {pretty_json}")
    print("\n\n")

KEY: 1f61bd83-c1cd-3658-8fab-29ba86d584a7
VALUE: {
  "statementId": "1f61bd83-c1cd-3658-8fab-29ba86d584a7",
  "statementDate": NaN,
  "annotations": [],
  "publicationDetails": {
    "publicationDate": "2018-12-20",
    "bodsVersion": "0.4",
    "license": "http://opendatacommons.org/licenses/pddl/1.0/",
    "publisher": {
      "name": "Extractive Industries Transparency initiative",
      "url": "https://eiti.org/open-data"
    }
  },
  "source": {
    "type": [
      "verified"
    ],
    "description": "",
    "url": "https://eiti.portaljs.com",
    "retrievedAt": "2024-05-18",
    "assertedBy": [
      {
        "name": NaN,
        "uri": NaN
      }
    ]
  },
  "declaration": "AF-20171221-20181220",
  "declarationSubject": "Afghanistan",
  "recordId": "1f61bd83-c1cd-3658-8fab-29ba86d584a7",
  "recordType": "entity",
  "recordDetails": {}
}



KEY: c40776d5-a273-3f9d-b805-075e804b9f3e
VALUE: {
  "statementId": "c40776d5-a273-3f9d-b805-075e804b9f3e",
  "statementDate": NaN,
  "an

# Part 2. Generating Entities

In [7]:
# test
filtered_df = pd.read_csv(url_part5)[lambda df: df['eiti_id_declaration'] == '1f61bd83-c1cd-3658-8fab-29ba86d584a7']

display(filtered_df.head())


/var/folders/z_/k9wh63yn4bzb0fzdgjkcx6tw0000gn/T/ipykernel_96606/273204525.py:2: DtypeWarning: Columns (5,8,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv(url_part5)[lambda df: df['eiti_id_declaration'] == '1f61bd83-c1cd-3658-8fab-29ba86d584a7']


,company_name,eiti_id_company,project_name,eiti_id_project,company_commodities,company_audited_financial statement_or_equivalent,company_public_listing_or_website,company_sector,company_id,company_type,...,in_kind_volume,in_kind_unit,comments,country,iso_alpha3_code,eiti_id_declaration,iso_alpha2_code,year,start_date,end_date
0,10000 EHDAS CONSTRUCTION AND GRAVEL COMPANY,9efb56c2-e978-40ec-985a-13c0b18c2413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private,...,NaN,n/v,2018-01-11,Afghanistan,AFG,1f61bd83-c1cd-3658-8fab-29ba86d584a7,AF,2018,2017-12-21,2018-12-20
1,ABAAN RAYAN LIMITED,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,NaN,NaN,coal,NaN,NaN,Other,9004655032,Private,...,NaN,n/v,NaN,Afghanistan,AFG,1f61bd83-c1cd-3658-8fab-29ba86d584a7,AF,2018,2017-12-21,2018-12-20
2,ABAAN RAYAN LIMITED,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,NaN,NaN,coal,NaN,NaN,Other,9004655032,Private,...,NaN,n/v,NaN,Afghanistan,AFG,1f61bd83-c1cd-3658-8fab-29ba86d584a7,AF,2018,2017-12-21,2018-12-20
3,ABAAN RAYAN LIMITED,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,NaN,NaN,coal,NaN,NaN,Other,9004655032,Private,...,NaN,n/v,NaN,Afghanistan,AFG,1f61bd83-c1cd-3658-8fab-29ba86d584a7,AF,2018,2017-12-21,2018-12-20
4,ABAAN RAYAN LIMITED,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,NaN,NaN,coal,NaN,NaN,Other,9004655032,Private,...,NaN,n/v,NaN,Afghanistan,AFG,1f61bd83-c1cd-3658-8fab-29ba86d584a7,AF,2018,2017-12-21,2018-12-20


In [8]:
# Extract unique companies, projects, and government entities
# todo: check if unique projects and unique government should be sourced from part3 and part4 respectively
unique_companies = df_part5[['company_name', 'eiti_id_company', 'iso_alpha2_code', 'country', 'company_public_listing_or_website', 'start_date', 'end_date']].dropna(subset=['eiti_id_company']).drop_duplicates()
unique_projects = df_part5[['project_name', 'eiti_id_project', 'iso_alpha2_code', 'country', 'start_date', 'end_date']].dropna(subset=['eiti_id_project']).drop_duplicates()
unique_government = df_part5[['government_entity', 'eiti_id_government', 'iso_alpha2_code', 'country', 'start_date', 'end_date']].dropna(subset=['eiti_id_government']).drop_duplicates()

# Add entity type column
unique_companies['entity_type'] = 'registeredEntity'
unique_projects['entity_type'] = 'arrangement'
unique_government['entity_type'] = 'stateBody'

# Combine unique entities into a single DataFrame
df_combined = pd.concat([unique_companies, unique_projects, unique_government], ignore_index=True)


In [9]:
# Initialize an empty DataFrame for df_entity
df_entity = pd.DataFrame()

# Apply the necessary transformations and keep only the transformed columns
df_entity['country_code'] = df_combined['iso_alpha2_code']
df_entity['name'] = df_combined['country']
df_entity['entity_type'] = df_combined['entity_type']

# Determine the subtype based on the government entity name
df_entity['subtype'] = df_combined.apply(
    lambda x: 'governmentDepartment' if x['entity_type'] == 'stateBody' and 'minist' in str(x['government_entity']).lower() else (
        'stateAgency' if x['entity_type'] == 'stateBody' else ''
    ), axis=1
)

# Identifiers and scheme code
df_entity['identifiers'] = df_combined.apply(
    lambda x: x['eiti_id_company'] if x['entity_type'] == 'registeredEntity' else (
        x['eiti_id_project'] if x['entity_type'] == 'arrangement' else (
            x['eiti_id_government'] if x['entity_type'] == 'stateBody' else ''
        )
    ), axis=1
)
df_entity['scheme_code'] = df_entity['identifiers']
df_entity['scheme_name'] = 'eiti'

# URI
df_entity['entity_uri'] = '/entity_statement/' + df_entity['identifiers'].fillna('')

# Fixed values
df_entity['is_component'] = 'false'

# Jurisdiction details
df_entity['country_or_subdivision_code'] = df_combined['iso_alpha2_code']

# Entity name based on the type
df_entity['entity_name'] = df_combined.apply(
    lambda x: x['company_name'] if x['entity_type'] == 'registeredEntity' else (
        x['project_name'] if x['entity_type'] == 'arrangement' else (
            x['government_entity'] if x['entity_type'] == 'stateBody' else ''
        )
    ), axis=1
)

# URI
df_entity['uri'] = df_combined['company_public_listing_or_website']

# Declaration reference
df_entity['declaration_reference'] = df_combined['iso_alpha2_code'] + '-' + df_combined['start_date'].str.replace('-','') + '-' + df_combined['end_date'].str.replace('-','')

# Display the df_entity DataFrame
display(df_entity.head())

,country_code,name,entity_type,subtype,identifiers,scheme_code,scheme_name,entity_uri,is_component,country_or_subdivision_code,entity_name,uri,declaration_reference
0,AF,Afghanistan,registeredEntity,,9efb56c2-e978-40ec-985a-13c0b18c2413,9efb56c2-e978-40ec-985a-13c0b18c2413,eiti,/entity_statement/9efb56c2-e978-40ec-985a-13c0...,false,AF,10000 EHDAS CONSTRUCTION AND GRAVEL COMPANY,NaN,AF-20171221-20181220
1,AF,Afghanistan,registeredEntity,,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,09acfc31-a64f-4e57-a9e2-79bd74bad8d2,eiti,/entity_statement/09acfc31-a64f-4e57-a9e2-79bd...,false,AF,ABAAN RAYAN LIMITED,NaN,AF-20171221-20181220
2,AF,Afghanistan,registeredEntity,,f78c41e6-827c-4751-a1d7-f7faa7ba5a25,f78c41e6-827c-4751-a1d7-f7faa7ba5a25,eiti,/entity_statement/f78c41e6-827c-4751-a1d7-f7fa...,false,AF,ABBAS GHAZNAVI LIMITED,NaN,AF-20171221-20181220
3,AF,Afghanistan,registeredEntity,,4a18fb8b-ec14-4e89-8f47-4beb9cf3edd7,4a18fb8b-ec14-4e89-8f47-4beb9cf3edd7,eiti,/entity_statement/4a18fb8b-ec14-4e89-8f47-4beb...,false,AF,ABDUL FATAH,n/v,AF-20171221-20181220
4,AF,Afghanistan,registeredEntity,,65d5fb0e-cb53-4d45-ba7d-0d66c20eecb2,65d5fb0e-cb53-4d45-ba7d-0d66c20eecb2,eiti,/entity_statement/65d5fb0e-cb53-4d45-ba7d-0d66...,false,AF,ABDUL RAOUF,n/v,AF-20171221-20181220


In [48]:
# Define the BODS entity structure based on the JSON schema
bods_entity_structure = {
    "isComponent": False,
    "entityType": {
        "type": "",
        "subtype": "",
        "details": ""
    },
    "name": "",
    "jurisdiction": {
        "name": "",
        "code": ""
    },
    "identifiers": [],
    "addresses": [],
    "uri": "",
    "publicListing": None,
    "formedByStatute": None
}

# Create a dictionary to hold the JSON files using declaration_reference as the key
entity_json_dict = {}

# Iterate over each row in df_entity to create JSON files
for index, row in df_entity.iterrows():
    if index % 100 == 0:  # Update progress every 100 rows
        print(f"\rProcessing row {index+1}/{len(df_entity)}", end='')

    entity_json = bods_entity_structure.copy()
    entity_json["isComponent"] = row['is_component']
    entity_json["entityType"]["type"] = row['entity_type']
    entity_json["entityType"]["subtype"] = row['subtype']
    entity_json["name"] = row['entity_name']
    entity_json["jurisdiction"]["name"] = row['name']
    entity_json["jurisdiction"]["code"] = row['country_code']
    entity_json["identifiers"] = [{
        "id": row['identifiers'],
        "scheme": row['scheme_code'],
        "schemeName": row['scheme_name'],
        "uri": row['entity_uri']
    }]
    entity_json["uri"] = row['uri']
    
    # Convert to JSON
    entity_json_str = json.dumps(entity_json, indent=2)
    
# Add the JSON string to the dictionary, grouping by declaration_reference
    if row['declaration_reference'] not in entity_json_dict:
        entity_json_dict[row['declaration_reference']] = []
    entity_json_dict[row['declaration_reference']].append(entity_json_str)

# Ensure the progress line is cleared after completion
print(f"\rProcessing completed. {len(df_entity)} rows processed.")

# Process data for DataFrame
data = [(key, len(value)) for key, value in entity_json_dict.items()]
df = pd.DataFrame(data, columns=['Declaration Reference', 'Number of JSON Objects'])
df_sorted = df.sort_values(by='Number of JSON Objects', ascending=False)

# Display the DataFrame
df_sorted


Processing completed. 8242 rows processed.


,Declaration Reference,Number of JSON Objects
25,GB-20190101-20191231,640
26,GB-20200101-20201231,629
24,GB-20180101-20181231,627
69,UA-20200101-20201231,505
42,MM-20170401-20180331,393
...,...,...
20,DO-20180101-20181231,10
23,ET-20160708-20170707,9
56,SR-20170101-20171231,8
57,SC-20170101-20171231,6


In [49]:
print(*entity_json_dict['SC-20180101-20181231'], sep='\n\n')

{
  "isComponent": "false",
  "entityType": {
    "type": "registeredEntity",
    "subtype": "",
    "details": ""
  },
  "name": "GX TECHNOLOGY",
  "jurisdiction": {
    "name": "Seychelles",
    "code": "SC"
  },
  "identifiers": [
    {
      "id": "dc174b45-bc3c-4dd2-b3dd-48fbc9034642",
      "scheme": "dc174b45-bc3c-4dd2-b3dd-48fbc9034642",
      "schemeName": "eiti",
      "uri": "/entity_statement/dc174b45-bc3c-4dd2-b3dd-48fbc9034642"
    }
  ],
  "addresses": [],
  "uri": NaN,
  "publicListing": null,
  "formedByStatute": null
}

{
  "isComponent": "false",
  "entityType": {
    "type": "registeredEntity",
    "subtype": "",
    "details": ""
  },
  "name": "JOGMEX",
  "jurisdiction": {
    "name": "Seychelles",
    "code": "SC"
  },
  "identifiers": [
    {
      "id": "21db233f-077c-4e7e-a9c5-a542fdd493ca",
      "scheme": "21db233f-077c-4e7e-a9c5-a542fdd493ca",
      "schemeName": "eiti",
      "uri": "/entity_statement/21db233f-077c-4e7e-a9c5-a542fdd493ca"
    }
  ],
  "add

In [ ]:
data = []
for var_name, json_str in entity_json_dict.items():
    # Load the JSON string as a Python list
    json_list = json.loads(json_str)
    # Count the number of items in the list
    count_json_objects = len(json_list)
    # Append to data list
    data.append((var_name, count_json_objects))

# Create DataFrame
df = pd.DataFrame(data, columns=['Key', 'Number of JSON Objects'])

# Sort the DataFrame by 'Number of JSON Objects' in descending order
df_sorted = df.sort_values(by='Number of JSON Objects', ascending=False)

# Display the DataFrame
df_sorted

# Part 3 - Matching statements to declarations

In [46]:
from collections import defaultdict

cleaned_entity_json_dict = {}
for declaration_reference, entity_json_str in entity_json_dict.items():
    cleaned_key = declaration_reference.replace("Declaration Reference: ", "").strip()
    cleaned_entity_json_dict[cleaned_key] = entity_json_str

# Create a dictionary to group entities by cleaned declaration_reference
grouped_entities = defaultdict(list)

# Iterate over each entry in cleaned_entity_json_dict to group entities by declaration_reference
for declaration_reference, entity_json_str in cleaned_entity_json_dict.items():
    entity_json = json.loads(entity_json_str)
    grouped_entities[declaration_reference].append(entity_json)

# Debug: Print the number of entities for each declaration reference
for declaration_reference, entities in grouped_entities.items():
    print(f"Declaration Reference: {declaration_reference} has {len(entities)} entities")

Declaration Reference: AF-20171221-20181220 has 1 entities
Declaration Reference: AF-20181221-20191220 has 1 entities
Declaration Reference: AL-20170101-20171231 has 1 entities
Declaration Reference: AL-20180101-20181231 has 1 entities
Declaration Reference: AR-20180101-20181231 has 1 entities
Declaration Reference: AM-20180101-20181231 has 1 entities
Declaration Reference: AM-20190101-20191231 has 1 entities
Declaration Reference: BF-20170101-20171231 has 1 entities
Declaration Reference: BF-20180101-20181231 has 1 entities
Declaration Reference: BF-20190101-20191231 has 1 entities
Declaration Reference: BF-20201201-20201231 has 1 entities
Declaration Reference: CI-20170101-20171231 has 1 entities
Declaration Reference: CI-20180101-20181231 has 1 entities
Declaration Reference: CM-20170101-20171231 has 1 entities
Declaration Reference: CD-20170101-20171231 has 1 entities
Declaration Reference: CG-20170101-20171231 has 1 entities
Declaration Reference: DE-20170101-20171231 has 1 entiti